In [ ]:
"""
Created on Tue Aug 30 15:35 2022

This script is to compute the RMSE based on point-by-point comparison for the melt near the grounding line for Figure E1. This produces the figure showing the results for the 50 km domain and offshore domain. I then combined 
the left panel of this figure with the right panel of Fig. 7.

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf

import seaborn as sns

import matplotlib as mpl
import glob

In [ ]:
%matplotlib qt5

In [ ]:
sns.set_context('paper')

READ IN DATA

In [ ]:
home_path='/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/generic_plots/'

In [ ]:
run_list = ['OPM006','OPM016','OPM018','OPM021']

diff_Gt_CVtime_list = []
diff_box1_CVtime_list = []
diff_Gt_CVisf_list = []
diff_box1_CVisf_list = []

ref_Gt_list = []
ref_box1_list = []


for n,nemo_run in enumerate(run_list):

    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
    
    # Geometry STUFF
    inputpath_data=home_path+'/DATA/BASAL_MELT_PARAM/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
    # make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
    map_lim = [-3000000,3000000]

    file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
    file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
    file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
    file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

    file_isf_conc = file_conc_cut['isfdraft_conc']
    
    ### REFERENCE MELT
    inputpath_boxes = home_path+'/DATA/BASAL_MELT_PARAM/interim/BOXES/nemo_5km_'+nemo_run+'/'
    box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
    box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')
    box_loc_config2 = box_charac_all_2D['box_location'].sel(box_nb_tot=box_charac_all_1D['nD_config'].sel(config=2))
    box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=0).drop('Nisf')
    NEMO_melt_rates_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO.nc')
    ref_box_1 = NEMO_melt_rates_2D['melt_m_ice_per_y'].where(box1==1).mean('time')
    ref_box1_mean = uf.weighted_mean(ref_box_1, ['x','y'], file_isf_conc)
    ref_box1_list.append(ref_box1_mean)
    
    ### READ IN THE PARAM FILES - CV TIME
    
    # Param files
    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
    melt_param_files_CVtime = list(sorted(glob.glob(outputpath_melt+'diff_melt_param_ref_box1_*_oneFRIS_CVtime.nc')))
    
    param_list = []
    for mfilename in melt_param_files_CVtime:
        paramname = mfilename[100:157].split('.')[0].split('_')[:-2]
        #paramname = mfilename[91:157].split('.')[0]
        #paramname = mfilename[105:157].split('.')[0].split('_')[:-1]
        mparam = '_'.join(paramname)
        param_list.append(mparam)
    
    ds_melt_param_CVtime  = xr.open_mfdataset(melt_param_files_CVtime, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    if 'option' in ds_melt_param_CVtime.coords:        
        ds_melt_param_CVtime = ds_melt_param_CVtime.drop('param').drop('option')
    else:
        ds_melt_param_CVtime = ds_melt_param_CVtime.drop('param')
    ds_melt_param_CVtime = ds_melt_param_CVtime.rename({'new_param': 'param'})
    ds_melt_param_CVtime = ds_melt_param_CVtime.assign_coords(param=param_list)
    diff_box1_CVtime = ds_melt_param_CVtime['melt_m_ice_per_y']
    squared_diff_box1_CVtime = uf.weighted_mean(diff_box1_CVtime**2, ['x','y'], file_isf_conc)
    diff_box1_CVtime_list.append(squared_diff_box1_CVtime)
    
    ### READ IN THE PARAM FILES - CV SHELVES
    
    melt_param_files_CVisf = list(sorted(glob.glob(outputpath_melt+'diff_melt_param_ref_box1_*_oneFRIS_CVshelves.nc')))
    ds_melt_param_CVisf  = xr.open_mfdataset(melt_param_files_CVisf, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    if 'option' in ds_melt_param_CVisf.coords:        
        ds_melt_param_CVisf  = ds_melt_param_CVisf.drop('param').drop('option')
    else:
        ds_melt_param_CVisf  = ds_melt_param_CVisf.drop('param')
    ds_melt_param_CVisf = ds_melt_param_CVisf.rename({'new_param': 'param'})
    ds_melt_param_CVisf = ds_melt_param_CVisf.assign_coords(param=param_list)
    diff_box1_CVisf = ds_melt_param_CVisf['melt_m_ice_per_y']
    squared_diff_box1_CVisf = uf.weighted_mean(diff_box1_CVisf**2, ['x','y'], file_isf_conc)
    diff_box1_CVisf_list.append(squared_diff_box1_CVisf)
        
# COMBINE THE SQUARED DIFFERENCE 
diff_box1_all_CVtime_sqmean = xr.concat(diff_box1_CVtime_list, dim='nemo_run')   
diff_box1_all_CVisf_sqmean = xr.concat(diff_box1_CVisf_list, dim='nemo_run')   
ref_box1_all_mean = xr.concat(ref_box1_list, dim='nemo_run')

In [ ]:
ref_box1_all_mean.mean()

In [ ]:
RMSE_CVtime = np.sqrt((diff_box1_all_CVtime_sqmean).mean('nemo_run')).load()
RMSE_CVisf = np.sqrt((diff_box1_all_CVisf_sqmean).mean('nemo_run')).load()

In [ ]:
param_list_of_int = ['linear_local', 'quadratic_local', 'quadratic_local_cavslope',
                    'quadratic_local_locslope', 'quadratic_mixed_mean','quadratic_mixed_cavslope',
                    'quadratic_mixed_locslope', 
                    'lazero19_2','lazero19_modif2',
                     'boxes_1_pismyes_picopno','boxes_2_pismyes_picopno','boxes_3_pismyes_picopno','boxes_4_pismyes_picopno',
                     'boxes_3_pismyes_picopyes', 'boxes_4_pismno_picopyes']

#param_list_of_int = ['linear_local', 'quadratic_local', 'quadratic_local_cavslope',
#                    'quadratic_local_locslope', 'quadratic_mixed_mean','quadratic_mixed_cavslope',
#                    'quadratic_mixed_locslope', 
#                    'lazero19_2']

param_simple_list = ['linear_local', 'quadratic_local', 'quadratic_local_cavslope',
                   'quadratic_local_locslope', 'quadratic_mixed_cavslope',
                   'quadratic_mixed_locslope', 'quadratic_mixed_mean']
param_plume_list = ['lazero19_2','lazero19_modif2']
param_box_list_hetero =  ['boxes_1_pismyes_picopno','boxes_2_pismyes_picopno','boxes_3_pismyes_picopno','boxes_4_pismyes_picopno'] #'boxes_1_pismno_picopno','boxes_2_pismno_picopno','boxes_3_pismno_picopno','boxes_4_pismno_picopno',
param_picop_list = ['boxes_3_pismyes_picopyes', 'boxes_4_pismno_picopyes']

param_list_box_whiskers = ['linear_local', 'quadratic_local', 'quadratic_local_cavslope',
                   'quadratic_local_locslope', 'quadratic_mixed_mean','quadratic_mixed_cavslope',
                   'quadratic_mixed_locslope', 
                    'lazero19_2',
                     'boxes_3_pismyes_picopno','boxes_4_pismyes_picopno',
                     'boxes_3_pismyes_picopyes', 'boxes_4_pismno_picopyes']

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/1.5,8.24*1.25/2),sharey=True)
for mparam in param_list_of_int[::-1]:
    
    if mparam in param_plume_list:
        ccolor = 'darkorange'
    elif mparam in param_box_list_hetero:
        ccolor = 'purple'
    elif mparam in param_picop_list:
        ccolor = 'maroon'
    elif mparam in param_simple_list:
        ccolor = 'gold'
        
    if 'profile_domain' in RMSE_CVtime.coords:
        axs[0].scatter(RMSE_CVtime.sel(param=mparam, profile_domain=50),mparam,marker='x',c='k', s=50)
        axs[1].scatter(RMSE_CVtime.sel(param=mparam, profile_domain=1000),mparam,marker='x',c='k', s=50)
        axs[0].scatter(RMSE_CVtime.sel(param=mparam, profile_domain=50),mparam,marker='o',c=ccolor, s=20)
        axs[1].scatter(RMSE_CVtime.sel(param=mparam, profile_domain=1000),mparam,marker='o',c=ccolor, s=20)

        axs[0].scatter(RMSE_CVisf.sel(param=mparam, profile_domain=50),mparam,marker='+',c='k', s=120)
        axs[1].scatter(RMSE_CVisf.sel(param=mparam, profile_domain=1000),mparam,marker='+',c='k', s=120)
        axs[0].scatter(RMSE_CVisf.sel(param=mparam, profile_domain=50),mparam,marker='o',c=ccolor, s=20)
        axs[1].scatter(RMSE_CVisf.sel(param=mparam, profile_domain=1000),mparam,marker='o',c=ccolor, s=20)

axs[0].set_xlim(0,3.5)
axs[1].set_xlim(0,2.05)

sns.despine()
#plt.savefig(plot_path+'comparison_box1_pointbypoint_50_1000.pdf')

In [ ]:
RMSE_CVtime